In [ ]:
import pandas as pd
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import re
import nltk

In [ ]:
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_excel('Input.xlsx')
df.head()


,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-...
1,2,https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/
2,3,https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/
3,4,https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-...
4,5,https://insights.blackcoffer.com/how-artificial-intelligence-can-boost-your-productivity-level/


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stopset = set(w.upper() for w in stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
articles = []
for x in df["URL"]:
    r = requests.get(x)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    articles.append(soup.get_text())

print(f'Total of {len(articles)} articles saved')

Total of 170 articles saved


In [ ]:
with open('StopWords_Generic.txt','r') as f:
    stop_words = f.read()

stop_words = stop_words.split('\n')
print(f'Total number of Stop Words are {len(stop_words)}')

#print(stop_words)

Total number of Stop Words are 121


In [ ]:
master_dict = pd.read_excel('LoughranMcDonald_MasterDictionary_2020.xlsx')
master_dict.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,312,1.422050e-08,1.335201e-08,3.700747e-06,96,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.367356e-10,8.882163e-12,9.362849e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,4.102067e-10,1.200533e-10,5.359747e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,15,6.836779e-10,4.080549e-10,1.406914e-07,14,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8009,3.650384e-07,3.798698e-07,3.523914e-05,1058,0,0,0,0,0,0,0,0,3,12of12inf


In [ ]:
positive_words = [x for x in master_dict[master_dict['Positive'] != 0]['Word']]

negative_words = [x for x in master_dict[master_dict['Negative'] != 0]['Word']]

print(f"Total positve words in dictionary are {len(positive_words)}")
print(f"Total negative words in dictionary are {len(negative_words)}")

Total positve words in dictionary are 354
Total negative words in dictionary are 2355


In [ ]:
def tokenize(text):
    text = re.sub(r'[^A-Za-z]',' ',text.upper())
    tokenized_words = word_tokenize(text)
    return tokenized_words

def remove_stopwords(words, stop_words):
    return [x for x in words if x not in stop_words]

def countscore(store, words):
    score = 0
    for x in words:
        if(x in store):
            score = score + 1
    return score

def sentiment(score):
    if(score < -0.5):
        return 'Most Negative'
    elif(score >= -0.5 and score < 0):
        return 'Negative'
    elif(score == 0):
        return 'Neutral'
    elif(score > 0 and score < 0.5):
        return 'Positive'
    else:
        return 'Very Positive'


def polarity(positive_score, negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score)+ 0.000001)


def subjectivity(positive_score, negative_score, num_words):
    return (positive_score+negative_score)/(num_words+ 0.000001)

def syllable_morethan2(word):
    if(len(word) > 2 and (word[-2:] == 'es' or word[-2:] == 'ed')):
        return False
    count =0
    vowels = ['a','e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1

    if(count > 2):
        return True
    else:
        return False

def syllable_count(word):
    if(word[-2:] == 'es' or word[-2:] == 'ed'):
        return 0
    count =0
    vowels = ['a','e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1
    return count

def fog_index_cal(average_sentence_length, percentage_complexwords):
    return 0.4*(average_sentence_length + percentage_complexwords)


In [ ]:
var = ['positive_score',
      'negative_score',
      'polarity_score',
       'subjectivity_score',
      'average_sentence_length',
      'percentage_of_complex_words',
      'fog_index',
       'average_no_of_words_per_sentence',
      'complex_word_count',
      'word_count',
       'syllable_per_word',
      'personal_pronoun',
      'average_word_length']


for v in var[:-1]:
        df[v] = 0.0

df[var[-1]] = 0.0

In [ ]:
df.head()

,URL_ID,URL,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_of_complex_words,fog_index,average_no_of_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronoun,average_word_length
0,1,https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,https://insights.blackcoffer.com/how-artificial-intelligence-can-boost-your-productivity-level/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lower_case(words):
    new_words = []
    for word in words:
        new_word = word.upper()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lower_case(words)
    words = remove_punctuation(words)
    return words

for i in range(len(articles)):
    text = re.sub('Item','ITEM',articles[i])
    text = re.sub('\[[^]]*\]', '', text)
    text = re.sub('[\d%/$]', '', text)
    tokenized_words = tokenize(text)
    tokenized_words = normalize(tokenized_words)
    #print(f'Total tokenized words are {len(tokenized_words)}')
    words = remove_stopwords(tokenized_words, stop_words)
    num_words = len(words)
    #print(f'Total words after removing stop words are {len(words)}')

    positive_score = countscore(positive_words, words)
    negative_score = countscore(negative_words, words)
    #print(f'Total positive score is {positive_score}')
    #print(f'Total negative score is {negative_score}')
    polarity_score = polarity(positive_score, negative_score)
    #print(polarity_score)
    subjectivity_score = subjectivity(positive_score, negative_score, num_words)
    #print(subjectivity_score)
    #print(sentiment(polarity_score))

    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    average_sentence_length = num_words/num_sentences
    #print(average_sentence_length)

    occ1 = re.findall("I",text,flags=re.IGNORECASE)
    occ2 = re.findall("WE",text,flags=re.IGNORECASE)
    occ3 = re.findall("OURS",text,flags=re.IGNORECASE)
    occ4 = re.findall("US",text,flags=re.IGNORECASE)
    personal_pronouns = len(occ1)+ len(occ2)+ len(occ3)+len(occ4)


    num_complexword =0
    syllablewords = 0
    charcount = 0
    for word in words:
        if(syllable_morethan2(word)):
            num_complexword = num_complexword+1
            #print(num_complexword)
        charcount = charcount+ len(word)
        #print(charcount)
        syllablewords =   syllablewords + syllable_count(word)
        #print(syllablewords)

    percentage_complexwords = num_complexword/num_words
    #print(percentage_complexwords)
    fog_index = fog_index_cal(average_sentence_length, percentage_complexwords)
    #print(fog_index)
    average_word_length = charcount/num_words



    df.at[i,'positive_score'] = positive_score
    df.at[i,'negative_score'] = negative_score
    df.at[i,'polarity_score'] = polarity_score
    df.at[i,'subjectivity_score'] = subjectivity_score
    df.at[i,'average_sentence_length'] = average_sentence_length
    df.at[i,'percentage_of_complex_words'] = percentage_complexwords
    df.at[i,'fog_index'] = fog_index
    df.at[i,'average_no_of_words_per_sentence'] = average_sentence_length
    df.at[i,'complex_word_count'] = num_complexword
    df.at[i,'word_count'] = num_words
    df.at[i,'syllable_per_word'] = syllablewords
    df.at[i,'personal_pronoun'] = personal_pronouns
    df.at[i,'average_word_length'] = average_word_length






In [ ]:
df.head()

,URL_ID,URL,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_of_complex_words,fog_index,average_no_of_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronoun,average_word_length
0,1,https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-...,0.0,1.0,-0.999999,0.076923,6.5,0.615385,2.846154,6.5,8.0,13.0,43.0,5.0,7.923077
1,2,https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/,0.0,1.0,-0.999999,0.076923,6.5,0.615385,2.846154,6.5,8.0,13.0,43.0,5.0,7.923077
2,3,https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/,0.0,1.0,-0.999999,0.076923,6.5,0.615385,2.846154,6.5,8.0,13.0,43.0,5.0,7.923077
3,4,https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-...,0.0,1.0,-0.999999,0.076923,6.5,0.615385,2.846154,6.5,8.0,13.0,43.0,5.0,7.923077
4,5,https://insights.blackcoffer.com/how-artificial-intelligence-can-boost-your-productivity-level/,0.0,1.0,-0.999999,0.076923,6.5,0.615385,2.846154,6.5,8.0,13.0,43.0,5.0,7.923077


In [ ]:
df.to_excel('output.xlsx')